In [ ]:
# %%
# filename: strat_backtestingpy_sma_showcase.ipynb

# --- 1. Imports and Setup ---
import numpy as np
import pandas as pd
import pandas_ta as ta
import talib
import tulipy as ti
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import EURUSD

# --- 2. Load the Sample Data ---
print("Loading built-in EURUSD sample data...")
data = EURUSD.copy()
print("Data loaded successfully.")

In [ ]:
# %%
# --- Example 1: Using TA-Lib ---
print("\n--- Showcase: TA-Lib ---")

def SMA(array, n):
    """Helper function to wrap talib.SMA for use with self.I"""
    return talib.SMA(array, n)

class SmaCrossTaLib(Strategy):
    fast_period = 10
    slow_period = 40

    def init(self):
        self.fast_sma = self.I(SMA, self.data.Close, self.fast_period)
        self.slow_sma = self.I(SMA, self.data.Close, self.slow_period)

    def next(self):
        if crossover(self.fast_sma, self.slow_sma):
            self.buy()
        elif crossover(self.slow_sma, self.fast_sma):
            self.position.close()

bt = Backtest(data, SmaCrossTaLib, cash=10000, commission=.002)
stats = bt.run()
print(stats)
bt.plot(resample=False)

In [ ]:
# %%
# --- Example 2: Using pandas-ta ---
print("\n--- Showcase: pandas-ta ---")

class SmaCrossPandasTa(Strategy):
    fast_period = 10
    slow_period = 40

    def init(self):
        self.fast_sma = self.I(ta.sma, pd.Series(self.data.Close), length=self.fast_period)
        self.slow_sma = self.I(ta.sma, pd.Series(self.data.Close), length=self.slow_period)

    def next(self):
        if crossover(self.fast_sma, self.slow_sma):
            self.buy()
        elif crossover(self.slow_sma, self.fast_sma):
            self.position.close()

bt = Backtest(data, SmaCrossPandasTa, cash=10000, commission=.002)
stats = bt.run()
print(stats)
bt.plot(resample=False)

In [ ]:
# %%
# --- Example 3: Using a Custom Python Function ---
print("\n--- Showcase: Custom Function ---")

def my_sma_func(price_series, n):
    return price_series.rolling(n).mean()

class SmaCrossCustom(Strategy):
    fast_period = 10
    slow_period = 40

    def init(self):
        self.fast_sma = self.I(my_sma_func, pd.Series(self.data.Close), self.fast_period)
        self.slow_sma = self.I(my_sma_func, pd.Series(self.data.Close), self.slow_period)

    def next(self):
        if crossover(self.fast_sma, self.slow_sma):
            self.buy()
        elif crossover(self.slow_sma, self.fast_sma):
            self.position.close()

bt = Backtest(data, SmaCrossCustom, cash=10000, commission=.002)
stats = bt.run()
print(stats)
bt.plot(resample=False)

In [ ]:
# %%
# --- Example 4: Using tulipy ---
print("\n--- Showcase: tulipy ---")

def tulip_sma(price_array, period):
    result = ti.sma(price_array, period=period)
    pad_size = len(price_array) - len(result)
    return np.concatenate([np.full(pad_size, np.nan), result])

class SmaCrossTulip(Strategy):
    fast_period = 10
    slow_period = 40

    def init(self):
        self.fast_sma = self.I(tulip_sma, self.data.Close, self.fast_period)
        self.slow_sma = self.I(tulip_sma, self.data.Close, self.slow_period)

    def next(self):
        if crossover(self.fast_sma, self.slow_sma):
            self.buy()
        elif crossover(self.slow_sma, self.fast_sma):
            self.position.close()

bt = Backtest(data, SmaCrossTulip, cash=10000, commission=.002)
stats = bt.run()
print(stats)
bt.plot(resample=False)